<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2024/ML2024_ex09notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex09notebookB

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2024)


----
## 準備
----


以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
seaborn.set()

----
## データの前処理(2) 特徴のスケーリング
----

「データの前処理(1)」に続いて，データの前処理の方法を解説します．
ここでは，量的データの値の範囲や散らばり方を適切に調整する **スケーリング** (scaling) という前処理を扱います．




---
### 特徴のスケーリング

#### 値の範囲や散らばり方が異なると...


機械学習の手法の多くは，特徴の値の範囲や散らばり方に影響を受けます．
そのことを理解するための具体例として，「身長(cm)」と「体重(kg)」という二つの値から，「人間」と「ほげ星人」を識別する問題を考えます．
「識別とは／最短距離法」の notebook などでも出てきたものです．

In [ ]:
# 「人間 vs ほげ星人」データの入手
dfHoge = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/humanvshoge.csv', header=0)
X = dfHoge.loc[:, ['height', 'weight']].to_numpy()
Y = np.zeros(X.shape[0])
hoge = dfHoge['label'] == 'Hoge'
Y[hoge] = 1 # Human = 0, Hoge = 1
dfHoge

In [ ]:
# 散布図を描く
fig, ax = plt.subplots(facecolor="white", figsize=(8, 4.8))
ax.set_xlim(0, 250)
ax.set_ylim(0,150)
ax.scatter(X[Y == 0, 0], X[Y == 0, 1], c='r', label='Human')
ax.scatter(X[Y == 1, 0], X[Y == 1, 1], c='b', label='Hoge')
ax.set_xlabel('Height', fontsize=16)
ax.set_ylabel('Weight', fontsize=16)
ax.legend(fontsize=16)
plt.show()

このデータでは，身長の値を[cm]の単位で，体重の値を[kg]の単位で表しています．
しかし，単位をどのように選ぶかは任意ですので，身長を[mm]や[m],[km]で表したり，体重を[g]や[t]で表したりすることも可能です．
このデータでは身長の値はだいたい $[50, 210]$ くらいの範囲にありますが，仮に[m]単位にすれば，$[0.5, 2.1]$ くらいの範囲に入ることになります．
その場合，体重に比べてずいぶん小さい値をとります．
あとで説明しますが，このように特徴値によって範囲や散らばり方が大きく異なっていると，機械学習の結果に悪影響が及ぶことがあります．


上では単位の違いを取り上げて説明しましたが，本質的には単位をどうとるかという問題ではありません．
以下では身長[cm]と体重[kg]のどちらかの値が $1/5$ となっている場合を図に描いてみることにします．

<img width="100%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/scaling.png">

上記の図の上段は，体重の値が元データの $1/5$ になった場合の散布図，下段は，身長の値が $1/5$ になった場合の散布図です．それぞれ，左側の図は元データと同じ描画範囲で描かれていますが，右側は，範囲も調整してあります．

左側の図は軸目盛の縦横の比率が1:1となっており，身長と体重の値の広がり方の違いがよく分かります．一方，右のように範囲を調整してしまうと，それが見えなくなっています．


上記の例で示しているような値の大きさの違いは，機械学習の結果に大きな影響を与えます．
そのことを確認するために，例として，元データ，体重を$1/5$にしたデータ，身長を$1/5$にしたデータ，のそれぞれを最短距離法で識別させてみます．プロトタイプはクラスごとのデータの平均とします．


<img width="100%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/scaling3.png">

上の図は，左から順に，元データ，体重を$1/5$にしたデータ，身長を$1/5$にしたデータの識別結果です．
値の大きさが変われば距離の値も変わり，大きな値ほど距離の値に大きく寄与するため，それぞれまるで異なる識別境界となっていることがわかります．

この識別の例に限らず，多くの機械学習の手法は，値の範囲や散らばり方の違いに影響を受けて結果が変わります．
そのため，値の範囲や散らばり方を適切に調整するスケーリングが重要となります．

#### 特徴のスケーリング

値の範囲や散らばり方を調整する方法として代表的なもの二つを紹介します．

一つ目は，「値の範囲をそろえる」というスケーリングの方法です．
特徴によって値の範囲が異なっているものを，一定の範囲（$[0, 1]$ や $[-1, 1]$ などがよく用いられる）にそろえます．
例えば，$[0, 1]$ にそろえる場合，学習データの特徴ごとに値の最小 $x_{\rm min}$ と最大 $x_{\rm max}$ を求めて，個々の値 $x_n$ を次式によって新しい値 $x_n^{\rm new}$ に変換します．
$$
x_n^{\rm new} = \frac{x_n - x_{\rm min}}{x_{\rm max} - x_{\rm min}}
$$
これによって，$[x_{\rm min},x_{\rm max}]$ だった範囲が $[0, 1]$ になります．


In [ ]:
# 値の範囲をそろえるスケーリング
Xmin = np.min(X, axis=0) # 身長体重それぞれの最小値
Xmax = np.max(X, axis=0) # 同最大値
print('元の最小値:' ,Xmin, '     元の最大値:', Xmax)
Xnew = (X - Xmin) / (Xmax - Xmin)
print('スケーリング後の最小値:' ,np.min(Xnew, axis=0), '     スケーリング後の最大値:', np.max(Xnew, axis=0))

# 散布図を描く
fig, ax = plt.subplots(facecolor="white", figsize=(6, 6))
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect('equal')
ax.scatter(Xnew[Y == 0, 0], Xnew[Y == 0, 1], c='r')
ax.scatter(Xnew[Y == 1, 0], Xnew[Y == 1, 1], c='b')
plt.show()

もう一つは，「値の平均を$0$に，分散を（標準偏差を）$1$にする」というスケーリングの方法です．
学習データの特徴ごとにその平均 $\mu$ と分散$\sigma^2$ を求めて，個々の値 $x_n$ を次式によって新しい値 $x_n^{\rm new}$ に変換します．
$$
x_n^{\rm new} = \frac{x_n - \mu}{\sigma}
$$
これによって，$x_n^{\rm new}$ は平均が $0$ で分散が $1$ になります．

In [ ]:
# 平均 0 分散 1 にするスケーリング
mu = np.mean(X, axis=0) # 身長体重それぞれの平均値
sigma = np.std(X, axis=0) # 同標準偏差
print('元の平均:' ,mu, '     元の標準偏差:', sigma)
Xnew = (X - mu) / sigma
print('スケーリング後の平均:' ,np.mean(Xnew, axis=0), '     スケーリング後の標準偏差:', np.std(Xnew, axis=0))

# 散布図を描く
fig, ax = plt.subplots(facecolor="white", figsize=(6, 6))
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_aspect('equal')
ax.scatter(Xnew[Y == 0, 0], Xnew[Y == 0, 1], c='r')
ax.scatter(Xnew[Y == 1, 0], Xnew[Y == 1, 1], c='b')
plt.show()

ここで説明した二つのスケーリング手法のうち，「値の範囲をそろえる」の方はよく **正規化**(normalization)と呼ばれます．
また，「値の平均を$0$に，分散を（標準偏差を）$1$にする」の方はよく **標準化**(staqndardization)と呼ばれます．
ただし，後者のことを正規化と呼ぶ場合もありますので注意が必要です．

正規化や標準化で用いる最大最小や平均分散の値は，学習データで計算するものです．学習データとは別のデータでテストする際には，学習データで求めた最大最小や平均分散を使って変換してやらねばなりません．さもなくば，学習データとテストデータで値の意味が違ってしまうことになります．実際のデータを扱う際にはこの点に注意が必要です．

#### ★★ やってみよう ★★

In [ ]:
dataL = np.array([20, -10, 30, 10,  50, 40])
print(dataL)

(1) 上記のセルを実行して表示される6つのデータに対して，値の範囲を $[0, 1]$ にするスケーリングを施す．
このとき，元のデータの $20$ という値はスケーリング後にいくつになるか答えなさい．

次のセルを実行すると答え合わせできます．

In [ ]:
data_min, data_max = np.min(dataL), np.max(dataL)
dataL_new = (dataL - data_min) / (data_max - data_min)
print(dataL_new)

(2) 上のデータを用いて決めたスケーリングの設定を用いて，以下に示すデータをスケーリングする．それぞれスケーリング後の値はいくつになるか答えなさい．


In [ ]:
dataT = np.array([20, -10, 50, 60, -20])
print(dataT)

次のセルを実行すると答え合わせできます．

In [ ]:
dataT_new = (dataT - data_min) / (data_max - data_min)
print(dataT_new)